# Overview

The purpose of this page is to give you some quick examples of what can be done with
SpectroChemPy.

See the [gallery of examples](examples/index.rst) and consult the
[user's guide](../userguide/introduction/introduction.ipynb)
for more information on using SpectroChemPy

Before using the package, we must load the **API (Application Programming Interface)**

In [ ]:
import spectrochempy as scp

## NDDataset, the main object

NDDataset is a python object, actually a container, which can represent most of your
multidimensional spectroscopic
data.

For instance, in the following we read data from a series of FTIR experiments,
provided  by the OMNIC software:

In [ ]:
ds = scp.read("irdata/nh4y-activation.spg")

### Display dataset information

Short information:

In [ ]:
print(ds)

Detailed information on the main metadata:

In [ ]:
ds

### Plotting a dataset

In [ ]:
_ = ds.plot()

### Slicing a dataset

In [ ]:
region = ds[:, 4000.0:2000.0]
_ = region.plot()

### Maths on datasets

In [ ]:
region.y -= region.y[0]  # make y coordinate relative to the first point
region.y.title = "time of dehydratation"
region -= region[-1]  # suppress the last spectra to all
_ = region.plot(colorbar=True)

### Processing a dataset

We just give here few examples

#### Smoothing

In [ ]:
smoothed = region.smooth(window_length=51, window="hanning")
_ = smoothed.plot(colormap="magma")

#### Baseline correction

In [ ]:
region = ds[:, 4000.0:2000.0]
smoothed = region.smooth(window_length=51, window="hanning")
blc = scp.Baseline()
blc.ranges = [[2000.0, 2300.0], [3800.0, 3900.0]]
blc.multivariate = True
blc.model = "polynomial"
blc.order = "pchip"
blc.n_components = 5

_ = blc.fit(smoothed)

In [ ]:
_ = blc.corrected.plot()

### Analysis

#### IRIS processing

In [ ]:
ds = scp.read_omnic("irdata/CO@Mo_Al2O3.SPG")[:, 2250.0:1950.0]
pressure = [
    0.00300,
    0.00400,
    0.00900,
    0.01400,
    0.02100,
    0.02600,
    0.03600,
    0.05100,
    0.09300,
    0.15000,
    0.20300,
    0.30000,
    0.40400,
    0.50300,
    0.60200,
    0.70200,
    0.80100,
    0.90500,
    1.00400,
]
ds.y = scp.Coord(pressure, title="Pressure", units="torr")
_ = ds.plot(colormap="magma")

In [ ]:

iris = scp.IRIS(reg_par=[-10, 1, 12])
K = scp.IrisKernel(ds, "langmuir", q=[-8, -1, 50])
iris.fit(ds, K)
_ = iris.plotdistribution(-7, colormap="magma")